In [1]:
import pandas as pd
import numpy as np
import spacy
import gensim
from sklearn.decomposition import LatentDirichletAllocation

C:\Users\krist\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('KPMG Tax Case - CSV_Summarized.csv')
df.head()

,Date,Title,Numac,Link FR,Link NL,Text,Cleaned Text,Summary_1,Summary_2
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020010053 BRUSSELS HOOFDSTEDELIJK GEWEST 8...,"Demissionair minister van Financiën, belast me...",NaN
1,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...,: 2020010214 MINISTERIE VAN DE FRANSE GEMEENS...,De Franse Gemeenschap en de Executieve van de ...,NaN
2,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020040138 FEDERALE OVERHEIDSDIENST FINANCI...,De ministers van Financiën en de staatssecreta...,NaN
3,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020020094,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020020094 FEDERALE OVERHEIDSDIENST FINANCI...,Het ministerie van Financiën heeft het ministe...,NaN
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministratio...,2020010193,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...,: 2020010193 FEDERALE OVERHEIDSDIENST FINANCI...,NaN,NaN


In [3]:
df['Summary_1'][11]

'Het ministerie van Financiën heeft een model van de aanvraag van certificaat en van certificaat bedoeld in artikel 442bis van het Wetboek van de inkomstenbelastingen 1992 zoals van toepassing op de onroerende voorheffing herzien.'

In [4]:
df.fillna('', inplace=True)

In [14]:
for idx, row in df.iterrows():

    nlp = spacy.load('nl_core_news_sm')
    text = df.loc[idx, 'Cleaned Text']
    tokenized = nlp(text)
    tokens = [token.text for token in tokenized]

    stopwords = spacy.lang.nl.stop_words.STOP_WORDS

    text_no_stop = [lemma for lemma in tokens if lemma not in stopwords]
    cleaned = ' '.join(text_no_stop)

    df.loc[idx,'Tag text'] = cleaned

In [ ]:
'''# TfidfVectorizer 

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# set of documents

train = 
test = 
# instantiate the vectorizer object


tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

# convert th documents into a matrix


tfidf_wm = tfidfvectorizer.fit_transform(train)

#retrieve the terms found in the corpora
# if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , it will give same output of get_feature_names() methods)
#count_tokens = tfidfvectorizer.get_feature_names() # no difference


tfidf_tokens = tfidfvectorizer.get_feature_names()

df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)


print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)
'''

In [ ]:
'''# Convert to document-term matrix
vectoriser = CountVectorizer(analyzer=preprocess_text)
example_matrix = vectoriser.fit_transform(df['Summary_1'])
# Extract feature/term names
feature_names = vectoriser.get_feature_names()
# Inspect document-term matrix
pd.DataFrame.sparse.from_spmatrix(example_matrix, 
                                  columns=feature_names)
'''

In [8]:
# Inspect topics
def describe_topics(lda, feature_names, top_n_words=7, show_weight=False):
    """Print top n words for each topic from lda model."""
    normalised_weights = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    for i, weights in enumerate(normalised_weights):  
        print(f"********** Topic {i+1} **********")
        if show_weight:
            feature_weights = [*zip(np.round(weights, 4), feature_names)]
            feature_weights.sort(reverse=True)
            print(feature_weights[:top_n_words], '\n')
        else:
            top_words = [feature_names[i] for i in weights.argsort()[:-top_n_words-1:-1]]
            print(top_words, '\n')

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object

vectorizer = TfidfVectorizer()


corpus = df['Tag text']
    
example_matrix = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
    
# print(tfidf_matrix.toarray())
    
# Inspect document-term matrix
    
pd.DataFrame.sparse.from_spmatrix(example_matrix, 
                                  columns=feature_names)
    
# Build lda model
lda = LatentDirichletAllocation(n_components=10, random_state=45)
lda.fit(example_matrix)
    
print(describe_topics(lda, feature_names, show_weight=True))


C:\Users\krist\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


********** Topic 1 **********
[(0.0006, 'paritair'), (0.0005, 'detailhandel'), (0.0004, 'subcomité'), (0.0004, 'magasin'), (0.0004, 'détail'), (0.0004, 'commerce'), (0.0003, 'université')] 

********** Topic 2 **********
[(0.0005, 'bedienden'), (0.0004, 'strafuitvoering'), (0.0004, 'diamant'), (0.0004, 'bbi'), (0.0003, 'triage'), (0.0003, 'teruggaaf'), (0.0003, 'paritair')] 

********** Topic 3 **********
[(0.0003, 'mobiliteitshulpmiddelen'), (0.0003, 'innoviris'), (0.0003, 'daspa'), (0.0003, 'commissariaat'), (0.0003, 'beschermingsmechanisme'), (0.0002, 'verstrekkers'), (0.0002, 'pers')] 

********** Topic 4 **********
[(0.0004, 'ouderschapsbescherming'), (0.0004, 'cateringdiensten'), (0.0003, 'restaurant'), (0.0003, 'gezinszorg'), (0.0003, 'alcoholische'), (0.0003, '414'), (0.0003, '342')] 

********** Topic 5 **********
[(0.0007, 'inlichting'), (0.0007, 'identificatienummer'), (0.0006, 'ontvangstmelding'), (0.0006, 'dergelijk'), (0.0005, 'wettig'), (0.0005, 'laboratoriumonderzoeken'

In [11]:
df['Tag text'][11]

'  : 2020040323 BRUSSELS HOOFDSTEDELIJK GEWEST 11 FEBRUARI 2020 . - Ministerieel besluit vaststelling modellen aanvraag certificaat certificaat bedoeld artikel 442bis Wetboek inkomstenbelastingen 1992 toepassing onroerende voorheffing    De Minister Brusselse Hoofdstedelijke Regering , belast Financiën Begroting , Gelet bijzondere wet 16 januari 1989 financiering Gemeenschappen Gewesten , artikel 5 ; Gelet Wetboek inkomstenbelastingen 1992 toepassing onroerende voorheffing , artikel 442bis ,   5 , gewijzigd ordonnantie 7 december 2017 houdende diverse bepalingen licht overname dienst onroerende voorheffing wijziging ordonnantie 21 december 2012 vaststelling fiscale procedure Brussels Hoofdstedelijk Gewest ; Gelet besluit Brusselse Hoofdstedelijke Regering 30 november 2017 aanstelling ambtenaren kader overname dienst onroerende voorheffing 1 januari 2018 , artikel 7 ; Gelet gelijke kansentest overeenkomstig artikel 2 ordonnantie 4 oktober 2018 invoering gelijke kansentest uitgevoerd 13 